# Trade Area Modeling
According to [Market Business News](https://marketbusinessnews.com/financial-glossary/trade-area-definition-meaning/), a trade (or market) area is the geographical area  where all or most of a business' sales volume occurs. It can be used to make decisions about the optimum location for business growth and expansion.

There are different trade area model and in this use case, we will be using the widely accepted [Huff model](https://en.wikipedia.org/wiki/Huff_model). The Huff model uses the **distance** between customers and existing business locations, or locations that are planned for the future, along with the **attractiveness of those locations**, to determine the likelihood of customers visiting those locations.

## Case Study
A beauty and personal care brand (we'll call them Lotionfy) has thrived as an exclusively online business for the past two years. They serve a predominantly US market, have done some market research and this is what they have discovered.
1. While the e-commerce share of total retail sales continues to rise, in-store shopping is still the preferred method for most US cusstomers and makes up a very large chunk of total retail sales. [[Reference](https://capitaloneshopping.com/research/online-vs-in-store-shopping-statistics/)]
2. More online shoppers are opting to pick up orders in-store and shoppers are generally returning to stores; retailers are reaching customers with small-format stores to encourage impulse purchases and reach new markets, among other benefits[[Reference](https://www2.deloitte.com/us/en/pages/consulting/articles/q1-2023-consumer-trends-report.html)]
3. Online businesses are developing physical footprints to keep their customers loyal and give them additional touchpoints for the brand; customers themselves now prefer omnichannel experiences [[Reference](https://www.inc.com/rebecca-deczynski/the-future-of-retail-isnt-direct-to-consumer-brands-embracing-brick-mortar-2023.html)]

In light of these, Lotionfy has decided to build their physical presence. To start, they have also decided to leverage partnerships with already existing [top department stores](https://www.junglescout.com/wp-content/uploads/2023/09/Jungle-Scout-Consumer-Trends-Report-Q3-2023.pdf) in the US like Walmart, Target, Kohl's etc. They will be leveraging the small-format store approach by drilling down to neighborhood locations.
They have decided to use a trade area model to know which cities to focus on and which store locations should be their areas of focus.

### Table of Contents
1. [Mining a customer base for the brand](#Mining-a-customer-base-for-the-brand)
    * [Modules and Clients](#Modules-and-Clients)
    * [Creating cities](#creating-cities)
    * [Creating city neighborhoods](#creating-city-neighborhoods)
    * [Obtaining neighborhood population](#obtaining-neighborhood-population)
    * [Generating customer population](#generating-customer-population)
    * [Find prime city with most customers](#find-prime-city-with-most-customers)
2. [Store Locations](#store-locations)
    * [Finding stores](#finding-stores)
    * [Distance matrix and travel time indices](#distance-matrix-and-travel-time-indices)
        * [Distance matrix](#distance-matrix)
        * [Travel time indices](#travel-time-indices)
    * [Store preferences](#store-preferences)
        * [Travel time](#travel-time)
        * [Notable commercial businesses](#notable-commercial-businesses)
        * [Highways](#highways)
        * [Design index](#design-index)
        * [Accessibility](#accessibility)
        * [Parking spaces](#parking-spaces)
        * [Store size](#store-size)
        * [Code](#code)

### Mining a customer base for the brand
To begin with, the [Statistical Atlas](https://statisticalatlas.com/United-States/Overview) website is scraped to get the list of some cities in the US. These cities are further scraped to get the list of city neighborhoods and their population. The map boundaries of these neighborhoods are gotten from [Zillow](https://catalog.data.gov/dataset/neighborhoods-us-2017-zillow-segs1).

It is quite unlikely for the entire population of a neighborhood to be part of the customer base so the customer population in the neighborhoods will be randomly generated, ensuring that it is no more than the total population. This is to avoid using confidential data from the business.

_Please note that this scraping was done in November/December 2023. After the time of this scraping, the website UI may have been changed by the web developers and the code may need to be changed accordingly._

#### Modules and Clients
Here, we load all Python modules/packages needed for the data mining. API keys were protected and stored using the dotenv module. The neighborhood map boundaries are stored using the Mercator projection so we also created a user-defined function to convert Mercator projections to lat-lng coordinates.

In [1]:
# import relevant packages
from shapely import wkt, Polygon, MultiPolygon
from timezonefinder import TimezoneFinder
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import googlemaps
import datetime
import requests
import polyline
import random
import pyproj
import pytz
import utm
import us
import os

In [2]:
# load environment variables
load_dotenv()
# create a random seed for reproducibility
random_seed = 42

In [3]:
# create Nominatim client
nominatim_locator = Nominatim(user_agent='trade_area_model', timeout=30)
# initialize googlemaps client - use your own key
gmaps = googlemaps.Client(key=os.getenv('google_api_key'))
# store OpenCage API key - use your own key
opencage_api_key = os.getenv('opencage_api_key')
# create timezone finder
tf = TimezoneFinder()

# convert from Mercator projection
mercator = pyproj.Proj(init='epsg:3857')
wgs84 = pyproj.Proj(init='epsg:4326')
def convert_to_latlon(geometry):
    shape = wkt.loads(geometry)
    if shape.geom_type == 'Polygon':
        return pyproj.transform(mercator, wgs84, *shape.exterior.coords.xy)
    elif shape.geom_type == 'MultiPolygon':
        new_polygons = []
        for polygon in shape.geoms:
            lng, lat = pyproj.transform(mercator, wgs84, *polygon.exterior.xy)
        new_polygons.append(Polygon(zip(lng, lat)))
        return MultiPolygon(new_polygons)
    else: return None

c:\Users\iolowoye\Anaconda3\envs\streamlit\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\iolowoye\Anaconda3\envs\streamlit\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


#### Creating cities
Here, we scrape a list of cities in the US (not all cities) and their states. We also use Nominatim to get the lat-lng coordinates of these cities. The results are stored in a csv file.

In [4]:
# create a list to store html list elements, and major cities and places
html_list, city_list = [], []
# initiate the http request to the website for scraping
url = f'https://statisticalatlas.com/United-States/Overview'
response = requests.get(url)
# if the http response is successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser') # parse html
    # find all div elements with the table class
    contents = soup.find_all('div', class_='info-table-contents-div')
    # if the hyperlink confirms that it is a city, add it to the html list
    for content in contents:
        hyperlink = content.find('a').get('href')
        if 'place' in hyperlink:
            cities = content.text.strip()
            html_list.append(cities)
            # add the cities in each html list element to the city list
            for cities in html_list:
                cities = cities.split(', ')
                city_list += cities
else: # if the http request failed
    print(f'Request failed: {response.status_code}')

In [5]:
# create a dictionary for cities and states
city_state_dict = {}
# for each city
for city in city_list:
    # geocode its location
    location = nominatim_locator.geocode(city).raw
    latitude, longitude = location['lat'], location['lon']
    state = location['display_name'].split(', ')[-2]
    # add the city, state, lat and long to the dictionary
    city_state_dict[city] = [state, latitude, longitude]

In [6]:
# create a dataframe of the customer cities
customer_cities = pd.DataFrame(city_state_dict.items(), columns=['city', 'state'])
customer_cities[['state','latitude','longitude']] = customer_cities['state'].apply(pd.Series)

In [7]:
# save customer cities data
customer_cities.to_csv('data/customers/customer_cities.csv', index=False)

#### Creating city neighborhoods
Here, the neighborhoods for all cities are scraped and cleaned. We restrict neighborhoods to only those who have available map boundaries and we convert these boundaries to lat-lng coordinates using the Mercator conversion function created earlier.

In [8]:
# create a city location column
customer_cities['location'] = customer_cities[['city','state']].agg(lambda x: ', '.join(x), axis=1)

In [9]:
# create a dictionary to store the list of neighborhoods in each city
city_neighborhoods = {}

# iterating through each row
for row in customer_cities.index:
    # store the state name, city and location in variables
    # rename certain cities based on the names used in the website we are web scraping from
    state = customer_cities.loc[row,'state'].replace(' ','-')
    city = customer_cities.loc[row,'city'].replace(' ','-')
    if city == 'Boise' and state == 'Idaho': city = 'Boise-City'
    elif city == 'Indianapolis' and state == 'Indiana': city = 'Indianapolis-city-(balance)'
    elif city == 'Louisville' and state == 'Kentucky': city = 'Louisville/Jefferson-County-metro-government-(balance)'
    elif city == 'Nashville-Davidson' and state == 'Tennessee': city = 'Nashville-Davidson-metropolitan-government-(balance)'
    location = customer_cities.loc[row,'location']
    # create the url to scrape from and initiate a http request
    url = f'https://statisticalatlas.com/place/{state}/{city}/Overview'
    response = requests.get(url)
    # if the http response is successful
    if response.status_code == 200:
        try: # try this
            soup = BeautifulSoup(response.text, 'html.parser') # parse html
            # find the div and class elements that should contain the list of neighborhoods
            # confirm from the hyperlinks that it contains neighborhood data
            contents = soup.find_all('div', class_='info-table-contents-div')
            for content in contents:
                hyperlink = content.find('a').get('href')
                if 'neighborhood' in hyperlink:
                    neighborhoods = content.text.strip()
                    # store the neighborhood data as values in the dictionary with the location as keys
                    city_neighborhoods[location] = neighborhoods
        except Exception as error: # if there is an error, print the error and continue execution
            print(url); print(str(error)); pass
    else: # if the response failed, print the status code
        print("Failed to get data, status code:",response.status_code)
    response.close()

In [10]:
# create a dataframe with the locations (city and state) and neighborhoods
city_communities = pd.DataFrame(list(city_neighborhoods.items()), columns=['location', 'neighborhoods'])

In [11]:
# split the neighborhoods by the delimiter
random.seed(random_seed)
city_communities['neighborhoods'] = city_communities['neighborhoods'].str.split(', ')#\
    #.apply(lambda x: random.sample(x, min(30, len(x))))
# explode the list of neighborhoods so each has its unique row
city_communities = city_communities.explode('neighborhoods', ignore_index=True)

In [12]:
# remove duplicates
city_communities = city_communities[~(city_communities.duplicated())]

In [13]:
# load the data with neighborhood boundaries
zillow_neighborhoods = gpd.read_file('data\ZillowNeighborhoods.gdb', layer=1)

In [14]:
# drop duplicates
zillow_neighborhoods.drop_duplicates('RegionID', inplace=True)

In [15]:
# create a dictionary of US state names and abbr
state_names_dict = {}
for state in us.STATES_AND_TERRITORIES:
    state_names_dict[state.abbr] = state
state_names_dict['DC'] = 'District of Columbia'

In [16]:
# convert state abbr to names in neighborhood boundaries data
zillow_neighborhoods['State'] = zillow_neighborhoods['State'].replace(state_names_dict)
# add county name for Nashville
zillow_neighborhoods.loc[zillow_neighborhoods['City'].str.contains('Nashville'), 'City'] = 'Nashville-Davidson'

In [17]:
# create a dataframe for neighborhood boundaries from Zillow
neighborhood_boundaries = pd.DataFrame([])
neighborhood_boundaries[['neighborhoods', 'geometry']] = zillow_neighborhoods[['Name','geometry']]
neighborhood_boundaries['location'] = zillow_neighborhoods[['City','State']].astype(str)\
    .apply(lambda x: ', '.join(x), axis=1)

In [18]:
# add neighborhood boundaries to city communities/neighborhoods data
city_communities = city_communities.merge(neighborhood_boundaries, how='inner', on=['location','neighborhoods'])
# convert geometry from Mercator projection to lng-lat coordinates
city_communities['geometry'] = city_communities['geometry'].astype(str).apply(convert_to_latlon)

C:\Users\iolowoye\AppData\Local\Temp\ipykernel_20336\1484609798.py:20: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lng, lat = pyproj.transform(mercator, wgs84, *polygon.exterior.xy)


In [19]:
# remove neighborhoods without boundary polygons
city_communities = city_communities[city_communities['geometry'].notna()]

In [20]:
# remove duplicates and cities without listed neighborhoods, if any
city_communities = city_communities[~(city_communities.duplicated()) & (city_communities['neighborhoods'] != '')]

#### Obtaining neighborhood population
Here, we scrape for the population within all neighborhoods. Some cities are bigger than others so for building the final customer base, we restrict the list of neighborhoods for each city to no more than thirty (30). Results are stored in csv files.

In [21]:
# explicitly cast the population column as a string
city_communities.loc[:,'population'] = ''

# iterating through each neighborhood
for row in city_communities.index:
    # store state name, city, and neighborhood in variables
    # replace certain cities based on the names in the website we are web scraping from
    location = city_communities.loc[row,'location'].replace(' ','-').split(',-')
    city, state = location[0], location[1]
    if city == 'Boise' and state == 'Idaho': city = 'Boise-City'
    elif city == 'Indianapolis' and state == 'Indiana': city = 'Indianapolis-city-(balance)'
    elif city == 'Louisville' and state == 'Kentucky': city = 'Louisville/Jefferson-County-metro-government-(balance)'
    elif city == 'Nashville-Davidson' and state == 'Tennessee': city = 'Nashville-Davidson-metropolitan-government-(balance)'
    neighborhood = city_communities.loc[row,'neighborhoods'].replace(' ','-').replace("'",'').replace('.','')
    # create the site url and initiate the http request
    url = f'https://statisticalatlas.com/neighborhood/{state}/{city}/{neighborhood}/Overview'
    response = requests.get(url)
    # if the request is successful
    if response.status_code == 200 and response.history == []:
        try: # try this
            soup = BeautifulSoup(response.text, 'html.parser') # parse html
            # find tr element containing the population number
            population_html = soup.find_all('tr')[-2]
            population = population_html.text.strip()
            # store population in its cell
            city_communities.loc[row, 'population'] = population
        except Exception as error: # for exceptions, print the url, error, and continue execution
            print(url, str(error), end=','); pass
    else: # if the http response fails, print its url, status code, and history
        print("Failed to get data from ", url, response.status_code, response.history)

Failed to get data from  https://statisticalatlas.com/neighborhood/Kentucky/Louisville/Jefferson-County-metro-government-(balance)/Algonquin/Overview 200 [<Response [303]>]
Failed to get data from  https://statisticalatlas.com/neighborhood/Kentucky/Louisville/Jefferson-County-metro-government-(balance)/Bon-Air/Overview 200 [<Response [303]>]
Failed to get data from  https://statisticalatlas.com/neighborhood/Kentucky/Louisville/Jefferson-County-metro-government-(balance)/Bowman/Overview 200 [<Response [303]>]
Failed to get data from  https://statisticalatlas.com/neighborhood/Kentucky/Louisville/Jefferson-County-metro-government-(balance)/Brownsboro-Zorn/Overview 200 [<Response [303]>]
Failed to get data from  https://statisticalatlas.com/neighborhood/Kentucky/Louisville/Jefferson-County-metro-government-(balance)/Chickasaw/Overview 200 [<Response [303]>]
Failed to get data from  https://statisticalatlas.com/neighborhood/Kentucky/Louisville/Jefferson-County-metro-government-(balance)/Edg

In [22]:
# filter out rows with empty population
city_communities = city_communities[city_communities['population'] != '']

In [23]:
# include a date key for versioning purposes
city_communities.loc[:, 'table_date'] = datetime.date.today()

In [24]:
# save this data in csv
city_communities.to_csv('data/all_city_communities.csv', index=False)

In [25]:
# set each city to have no more than 30 neighborhoods/communities
# this step can be skipped
city_communities = city_communities.groupby('location', group_keys=False).apply(lambda x: x.sample(n=min(30, len(x)),
                                                                                       random_state=random_seed))

In [26]:
# save this data in csv
city_communities.to_csv('data/city_communities.csv', index=False)

#### Generating customer population
Here, we randomly generate the customer base within all neighborhoods ensuring that the customer base is only within 10% to 60% of the total neighborhood population and the result is stored in a csv file. Remember that this is done to protect confidential information.

In a business use case, the KYC data will be used to know the customer base of a brand and where they reside.

In [27]:
# create a new column for customer population with reproducible random values less than the total population
np.random.seed(random_seed)
customer_communities = city_communities.copy()
customer_communities['population'] = (customer_communities['population'].str.replace(',','').astype(float)\
                                      * np.random.uniform(0.1, 0.6, len(customer_communities))).astype(int)

In [28]:
# save this data in csv
customer_communities.to_csv('data/customers/customer_communities.csv', index=False)

#### Find prime city with most customers
Here, we find the city with the biggest customer base. The trade area model will be built within this city.

In [29]:
# find city location with the most customers
number_of_customers = customer_communities.groupby('location')['population'].sum()\
    .sort_values(ascending=False)
number_of_customers

location
Phoenix, Arizona                    514656
Los Angeles, California             332142
New York, New York                  329176
San Jose, California                326346
Dallas, Texas                       282115
Houston, Texas                      209220
Boston, Massachusetts               208396
Fresno, California                  197803
Memphis, Tennessee                  186313
Colorado Springs, Colorado          183442
Las Vegas, Nevada                   162448
Chicago, Illinois                   158123
Arlington, Texas                    130606
Raleigh, North Carolina             129287
El Paso, Texas                      127429
Miami, Florida                      124829
Indianapolis, Indiana               122343
Cleveland, Ohio                     108794
Virginia Beach, Virginia            103860
Long Beach, California               98455
Philadelphia, Pennsylvania           89448
San Diego, California                88608
San Francisco, California            86354
De

In [30]:
# select prime city location
prime_location = number_of_customers.index[0]
# get community coordinates
customer_communities['coord'] = customer_communities['geometry'].apply(lambda x: wkt.loads(x).centroid.xy)
# select customer neighborhoods in the prime city location
prime_city_neighborhoods = customer_communities.loc[customer_communities['location'] == prime_location,
                                                ['location','neighborhoods']]

### Store Locations
The following department stores in the prime location will be used as potential store locations: Walmart, Target, JCPenny, Kohl's and Marshalls

#### Finding stores
Here, we get some locations of the selected department stores using the Google Maps API and store the results in a csv file.

In [31]:
def get_address(geometry):
    lat, lng = geometry.centroid.y, geometry.centroid.x
    result = gmaps.reverse_geocode((lat, lng))
    address = result[0]['formatted_address']
    address = ', '.join(address.split(', ')[:2])
    return address

In [32]:
def get_nominatim_address(gmap_address):
    results = gmaps.places(query=gmap_address)
    geom = results['results'][0]['geometry']['location']
    lat, lng = geom['lat'], geom['lng']
    address = nominatim_locator.reverse((lat, lng)).raw['display_name']
    return address

In [33]:
# check for these department stores on GMaps
# get the OSM features of these stores
department_stores = pd.DataFrame()
dept_store_list = ["Walmart", "Target", "JCPenney", "Kohl's", "Marshalls"]
for name in dept_store_list:
    query = f"{name}, {prime_location}"
    results = gmaps.places(query=query)
    for result in results['results']:
        address = result['formatted_address']
        name = result['name']
        try:
            nominatim_address = get_nominatim_address(address)
            df = ox.features_from_address(nominatim_address, tags = {'shop':['department_store','supermarket']})
            df.loc[:,['gmap_name', 'address']] = name, ', '.join(address.split(', ')[:2])
            department_stores = pd.concat([department_stores, df], axis=0)
        except Exception as error: 
            print(f'Error with {name}; {address}: {error}', end='\n\n'); pass

Error with Target; 5715 N 19th Ave, Phoenix, AZ 85015, United States: Nominatim could not geocode query 'Target, 5715;1760, West Montebello Avenue, Alhambra, Phoenix, Maricopa County, Arizona, 85015, United States'

Error with Target Grocery; 5715 N 19th Ave, Phoenix, AZ 85015, United States: Nominatim could not geocode query 'Target, 5715;1760, West Montebello Avenue, Alhambra, Phoenix, Maricopa County, Arizona, 85015, United States'

Error with Kohl's; 34830 N Vly Pkwy, Phoenix, AZ 85086, United States: No data elements in server response. Check log and query location/tags.



In [34]:
# list all available department store names
department_store_names = department_stores['name'].dropna().unique().tolist()
# select all stores that contain the target store names
selected_stores = [item for item in department_store_names if any(store in item for store in dept_store_list)]
# keep stores with defined names and geopolygons
department_stores = department_stores.loc[(department_stores['name'].isin(selected_stores)) &
                                          (department_stores['geometry'].astype(str).str.contains('POLYGON')),
                                          ['name','gmap_name','address', 'geometry','building:levels']]
# remove duplicates
department_stores = department_stores.sort_index().reset_index().drop_duplicates('osmid',keep='last')

In [35]:
# get the correct gmaps address of stores near to the gmap store that was also selected by OSM
department_stores.loc[department_stores['name'] != department_stores['gmap_name'], 'address'] = \
    department_stores['geometry'].apply(get_address)
# remove the gmap name from the dataframe
department_stores.drop('gmap_name', axis=1, inplace=True)

In [36]:
# create store locations with name and address
department_stores['store_location'] = department_stores[['name','address']].apply(lambda x:', '.join(x), axis=1)
store_locations = department_stores['store_location'].unique()

In [37]:
department_stores['city_location'] = prime_location

In [38]:
department_stores.to_csv('data/stores/store_data.csv', index=False)

#### Distance matrix and travel time indices
Here, for each store location, we leverage the Google Maps API to estimate:
1. The distance between the customer neighborhoods and the store
2. The time spent in rush-hour traffic (at 5pm)
3. The route/path from the neighborhoods to the store

In [39]:
# get timezone of prime location
geocode_prime_location = gmaps.geocode(prime_location)
prime_location_geom = geocode_prime_location[0]['geometry']['location']
timezone = tf.timezone_at(lat=prime_location_geom['lat'], lng=prime_location_geom['lng'])

In [40]:
for store in store_locations:
    # create column for store path, distance and time from neighborhood
    prime_city_neighborhoods.loc[:, 'distance '+store] = ''
    prime_city_neighborhoods.loc[:, 'time '+store] = ''
    prime_city_neighborhoods.loc[:, 'path '+store] = ''
    # iterating through each neighborhood
    for row in prime_city_neighborhoods.index:
        # store the customer location and its geolocation in variables
        neighborhoods = prime_city_neighborhoods.loc[row, 'coord']
        lng, lat = neighborhoods[0][0], neighborhoods[1][0]
        # set gmaps departure time
        try:
            departure_time = datetime.datetime.now(pytz.timezone(timezone)).replace(hour=17, minute=0, second=0)
            # get the path, distance and travel time and add them to the neighborhood df
            geocode_result = gmaps.directions((lat, lng), store, mode='driving', traffic_model='pessimistic',
                                              units='metric', departure_time=departure_time)
        except:
            day = datetime.date.today().day + 1
            departure_time = datetime.datetime.now(pytz.timezone(timezone)).replace(day=day, hour=17, minute=0, second=0)
            # get the path, distance and travel time and add them to the neighborhood df
            geocode_result = gmaps.directions((lat, lng), store, mode='driving', traffic_model='pessimistic',
                                            units='metric', departure_time=departure_time)
        dist_duration = geocode_result[0]['legs'][0]
        distance = dist_duration['distance']['text']
        time = dist_duration['duration_in_traffic']['text']
        encoded_path = geocode_result[0]['overview_polyline']['points']
        path = polyline.decode(encoded_path)
        prime_city_neighborhoods.loc[row, 'distance '+store] = distance
        prime_city_neighborhoods.loc[row, 'time '+store] = time
        prime_city_neighborhoods.loc[row, 'path '+store] = str(path)

In [41]:
# create a function to convert distances  to km
def convert_to_km(distance_str):
    value, unit = float(distance_str.split()[0]), distance_str.split()[1].lower()
    unit_conversion_dict = {'km':1.0, 'm':0.001}
    return value * unit_conversion_dict[unit]

In [42]:
# create a function to convert time to minutes
def convert_to_mins(time_str):
    total_minutes = 0
    time = time_str.split()
    if 'hours' in time or 'hour' in time: total_minutes += (int(time[0]) * 60) + int(time[2])
    elif 'mins' in time or 'min' in time: total_minutes += int(time[0])
    return total_minutes

##### Distance matrix

In [43]:
# columns for distance matrix
distance_columns = ['neighborhoods'] + [col for col in prime_city_neighborhoods.columns if 'distance' in col]
# create distance matrix
distance_matrix = prime_city_neighborhoods.loc[:, distance_columns]
# remove neighborhood prefixes and suffixes
distance_matrix.columns = distance_matrix.columns.str.removeprefix('distance').str.removesuffix(prime_location).str.strip()\
    .str.removesuffix(',')
# set neighborhoods as index
distance_matrix.set_index('neighborhoods', inplace=True)
# standardize units
for col in distance_matrix.columns:
    try: distance_matrix[col] = distance_matrix[col].map(convert_to_km)
    except Exception as error: print(f'Could not convert for column {col}'); pass

In [44]:
# save data
distance_matrix.to_csv('data/stores/store_distances.csv')

##### Travel time indices

In [45]:
# columns for travel time
time_columns = ['neighborhoods'] + [col for col in prime_city_neighborhoods if 'time' in col]
# create travel time data
travel_time = prime_city_neighborhoods.loc[:, time_columns]
# remove neighborhood prefixes and suffixes
travel_time.columns = travel_time.columns.str.removeprefix('time').str.removesuffix(prime_location).str.strip()\
    .str.removesuffix(',')
# set neighborhoods as index
travel_time.set_index('neighborhoods', inplace=True)
# standardize units
for col in travel_time.columns:
    try: travel_time[col] = travel_time[col].map(convert_to_mins)
    except Exception as error: print(f'Could not convert for column {col}'); pass

In [46]:
# store data
travel_time.to_csv('data/stores/traffic_time_mins.csv')

##### Path

In [47]:
# columns for travel time
path_columns = ['neighborhoods'] + [col for col in prime_city_neighborhoods if 'path' in col]
# create travel time data
path_df = prime_city_neighborhoods.loc[:, path_columns]
# remove neighborhood prefixes and suffixes
path_df.columns = path_df.columns.str.removeprefix('path').str.removesuffix(prime_location).str.strip()\
    .str.removesuffix(',')
# set neighborhoods as index
path_df.set_index('neighborhoods', inplace=True)

In [48]:
# store data
path_df.to_csv('data/stores/path_to_store.csv')

#### Store preferences
These are metrics used to determine how attractive the store locations are to the customers. These include the travel time indices, number of notable commercial businesses, number of highways, design index, accessibility (how many drivable and walkable roads are there?), number of available parking spaces, and size of the store.

**For store preference metrics other than travel time indices, the Open Street Map network is leveraged to obtain features within a 1km radius for each store.**

##### Travel time
The average travel time to each store from all neighborhoods is used as the traffic index to guage the willingness of customers to drive/bike/walk to the store. The higher the travel time, the less willing customers will be to go to the store, especially since this is not for a luxury brand.

##### Notable commercial businesses
These are named commercial districts/buildings near the store. The presence of these could indicate that customers would have other reasons to be in that location, especially during work hours (if they work in those buildings, they can stop by the store during break hours or after hours)

##### Highways
These are major (motorway, trunk, primary, secondary, motorway link, trunk link, primary link, secondary link, motorway junction) roads near the store. If there are more major roads leading to the location, the easier it is to drive there.

##### Design index
The presence of these specific features were used to determine the design index: cafes, restaurants, fuel stations, taxi stops, atms, banks, hospitals, community centres, conference centres, events venues, exhibition venues, marketplaces, resorts, fitness centers, gardens, parks, beaches, office buildings, educational buildings, religious buildings, hotels, and tourist attractions.

All of these make places more attractive to people in general, and customers could have other reasons to visit that store location.

##### Accessibility
In this use case, this speaks to the presence of minor roads to drive through, bus stops, subway stations, walkways, and cycle roads. These features make it even easier to get to the location.

##### Parking spaces
Each department store will have its own parking lot. The presence of another public parking space nearby will also be advantageous. If the store parking lot is full, there's less need to worry about where to park without attracting a parking fine.

##### Store size
The bigger the store, the more variety they would be able to sell, and the more likely they are to have everything on a customer's shopping list.

##### Code

In [49]:
# create travel index of store
store_preferences = round(travel_time.mean(numeric_only=True), 1).to_frame(name='avg_travel_time')

In [50]:
# create a list of major highway tags
major_highways = ['motorway', 'trunk', 'primary', 'secondary', 'motorway_link', 'trunk_link',
                  'primary_link', 'secondary_link', 'motorway_junction']

# store the city and state in variables
store_city = prime_location.split(', ')[0].replace(' ','-')
store_state = prime_location.split(', ')[1].replace(' ','-')

# create placeholder columns
store_preferences.loc[:, 'business_communities'] = 0
store_preferences.loc[:, 'highways'] = 0
store_preferences.loc[:, 'design'] = 0
store_preferences.loc[:, 'accessibility'] = 0
store_preferences.loc[:, 'parking_space'] = 1
store_preferences.loc[:, 'store_size'] = 0.0

# create dictionaries for the different dataframes
businesses_dict, highways_dict, design_dict, road_dict, walkways_dict, parking_dict = ({} for _ in range(6))

# iterate through the store locations
for store in store_locations:
    
    store_address = get_nominatim_address(store)
    """ BUSINESS DISTRICTS """
    # find and count number of commercial landspace
    try:
        businesses_df = ox.features_from_address(store_address, tags={'landuse':'commercial'})
        businesses_dict[store] = businesses_df
        businesses = businesses_df['name'].nunique()
        store_preferences.loc[store, 'business_communities'] = businesses
    except: pass
    
    """ HIGHWAYS """
    # find and count number of highways
    try:
        highways_df = ox.features_from_address(store_address, tags={'highway':major_highways})
        highways_dict[store] = highways_df
        highways = highways_df['name'].nunique()
        store_preferences.loc[store, 'highways'] = highways
    except: pass
    
    """ MERCHANDIZING DESIGN """
    # find amenities and other neighborhood design touchpoints
    try:
        design_df = ox.features_from_address(store_address, tags={'amenity':['cafe','restaurant','fuel','taxi','atm','bank','hospital','community_centre','conference_centre','events_venue','exhibition_venue','marketplace'], 
                                                                  'leisure':['beach_resort','fitness_centre','garden','park'],
                                                                  'natural':'beach', 'office':True,
                                                                  'landuse':['education','religious'],
                                                                  'tourism':['attraction','hotel']})
        design_dict[store] = design_df
        design = design_df.name.nunique()
        store_preferences.loc[store, 'design'] = design
    except: pass
    
    """ ACCESSIBILITY """
    # estimate accessibility
    try:
        road_df = ox.features_from_address(store_address, tags={'highway':['tertiary', 'bus_stop'], 
        'railway':'subway_entrance'})
        road_dict[store] = road_df
        roads = road_df.name.nunique()
    except: roads = 0; pass
    try:
        walkways_df = ox.features_from_address(store_address, tags={'highway':['footway', 'track','cycleway']}).reset_index()
        walkways_dict[store] = walkways_df
        walkways = walkways_df.osmid.nunique()
    except: walkways = 0; pass
    store_preferences.loc[store, 'accessibility'] = roads + walkways
    
    """ PARKING SPACE """
    # create dataframe to store parking space
    try:
        parking_df = ox.features_from_address(store_address, tags = {'building':'parking', 'amenity':'parking'})
        parking_df = parking_df.loc[parking_df['access'] == 'yes', :].reset_index().drop_duplicates('osmid')
        try: parking_df['building:levels'] = parking_df['building:levels'].fillna(1).astype(int)
        except: parking_df['building:levels'] = 1
        parking_dict[store] = parking_df
        parking = parking_df['building:levels'].sum(numeric_only=True) + 1
        if parking > 0: store_preferences.loc[store, 'parking_space'] = parking
    except: pass
    
    """ STORE SIZE """
    # select geometry
    store_size_df = department_stores.loc[department_stores['store_location']==store, :]
    try: geometry = list(store_size_df['geometry'].iloc[0].exterior.coords)
    except: geometry = list(wkt.loads(store_size_df['geometry'].iloc[0]).exterior.coords)
    # reverse the geometry position to be lat-long not long-lat
    geometry_reverse = []
    for coord in geometry:
        latlon = (coord[1], coord[0])
        geometry_reverse.append(latlon)
    # extract utm zone
    zone = utm.from_latlon(geometry_reverse[0][0], geometry_reverse[0][1])
    zone_num = zone[2]
    # get utm coordinates of store geometry
    utm_zone = pyproj.Proj(f"+proj=utm +zone={zone_num} +ellps=WGS84 +units=m")
    utm_coord = [utm_zone(latitude=lat, longitude=long) for lat, long in geometry_reverse]
    # calculate the area of the selected retail store
    area_m2 = Polygon(utm_coord).area
    # count number of building levels
    try: bld_lvls = int(store_size_df['building:levels'].iloc[0])
    except: bld_lvls = 1
    # calculate total store area
    store_size_m2 = round(area_m2 * bld_lvls, 2)
    # add to the store preferences data
    store_preferences.loc[store, 'store_size'] = store_size_m2
    print(f'{store} preferences completed')

Walmart Neighborhood Market, 115 E Dunlap Ave, Phoenix preferences completed
Walmart Supercenter, 4747 E Cactus Rd, Phoenix preferences completed
Target, 21001 N Tatum Blvd, Phoenix preferences completed
Target, 7409 W Virginia Ave, Phoenix preferences completed
Walmart Supercenter, 1825 W Bell Rd, Phoenix preferences completed
Walmart Supercenter, 5250 W Indian School Rd, Phoenix preferences completed
Target, 9830 W Lower Buckeye Rd, Tolleson preferences completed
Kohl's, 3000-3010 S 99th Ave, Tolleson preferences completed
Walmart Supercenter, 3721 E Thomas Rd, Phoenix preferences completed
Walmart Supercenter, 6145 N 35th Ave, Phoenix preferences completed
Walmart Supercenter, 7575 W Lower Buckeye Rd, Phoenix preferences completed
Walmart Supercenter, 2020 N 75th Ave, Phoenix preferences completed
Target, 4722 E Ray Rd #3, Phoenix preferences completed
Walmart Supercenter, 6150 S 35th Ave, Phoenix preferences completed
Kohl's, 17232 N 19th Ave, Phoenix preferences completed
Walmart 

In [51]:
# save preferences
store_preferences.to_csv('data/stores/store_preferences.csv', index_label='store')
# save features dataframes
clear_df = pd.DataFrame([])
for key in businesses_dict:
    businesses_dict[key].loc[:, 'store'] = key
    clear_df = pd.concat([clear_df, businesses_dict[key]], axis=0)
    clear_df.to_csv('data/stores/business_communities.csv')
clear_df = pd.DataFrame([])
for key in highways_dict:
    highways_dict[key].loc[:, 'store'] = key
    clear_df = pd.concat([clear_df, highways_dict[key]], axis=0)
    clear_df.to_csv('data/stores/highways.csv')
clear_df = pd.DataFrame([])
for key in design_dict:
    design_dict[key]['store'] = key
    clear_df = pd.concat([clear_df, design_dict[key]], axis=0)
    clear_df.to_csv('data/stores/attractions.csv')
clear_df = pd.DataFrame([])
for key in road_dict:
    road_dict[key]['store'] = key
    clear_df = pd.concat([clear_df, road_dict[key]], axis=0)
    clear_df.to_csv('data/stores/roads.csv')
clear_df = pd.DataFrame([])
for key in walkways_dict:
    walkways_dict[key]['store'] = key
    clear_df = pd.concat([clear_df, walkways_dict[key]], axis=0)
    clear_df.to_csv('data/stores/walkways.csv', index=False)
clear_df = pd.DataFrame([])
for key in parking_dict:
    parking_dict[key]['store'] = key
    clear_df = pd.concat([clear_df, parking_dict[key]], axis=0)
    clear_df.to_csv('data/stores/parking_spaces.csv', index=False)